In [ ]:
import tensorflow as tf
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
strategy = tf.distribute.TPUStrategy(tpu)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import pandas as pd
import numpy as np
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from google.colab import drive

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
#Connecting to Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Loading the dataset from Google Colab
df = pd.read_csv('/content/drive/MyDrive/yuksekTez/airline_dataset/cleaned_text.csv')

In [ ]:
# Create the feature matrix and target vector
X = df['clean_text1'].values
y = df['airline_sentiment'].values

In [ ]:
df["clean_text1"] = df["clean_text1"].apply(str)


In [ ]:
# Split the data into training and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
with strategy.scope():
  # Extract features from the preprocessed data using bag-of-words
  vectorizer = CountVectorizer()
  X_train = vectorizer.fit_transform(X_train)
  X_test = vectorizer.transform(X_test)

In [ ]:
with strategy.scope():
  # Train the Naive Bayes classifier
  model = MultinomialNB()
  model.fit(X_train, y_train)

In [ ]:
# Test the classifier on the test set
y_pred = model.predict(X_test)

In [ ]:
# Evaluate the performance of the classifier
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

    negative       0.79      0.96      0.87      1889
     neutral       0.70      0.38      0.49       580
    positive       0.82      0.59      0.69       459

    accuracy                           0.79      2928
   macro avg       0.77      0.64      0.68      2928
weighted avg       0.78      0.79      0.76      2928

[[1807   57   25]
 [ 324  221   35]
 [ 150   37  272]]
